## Preconfiguration

### Importing packages & functions

In [ ]:
using Pkg
Pkg.activate("/home/qingyanz/AtomicGraphNets.jl")

In [ ]:
using CSV
using DataFrames
using SparseArrays
using Random, Statistics
using Flux
using Flux: @epochs
# using GeometricFlux
using SimpleWeightedGraphs
using AtomicGraphNets
using DelimitedFiles
using Distributed
using ProgressBars
using ChemistryFeaturization
using Serialization
import Dates

In [ ]:
using PyCall

# s = pyimport("pymatgen.core.structure")

In [ ]:
import Flux:@functor

# import CrystalGraphConvNets.inverse_square
# include("../../CrystalGraphConvNets.jl/src/graph_functions.jl")

In [ ]:
#= Before defining the model...
  All floats in the CrystalGraphConvNets.jl package are defined as Float32. 
  However certain loss functions given by Flux yield Float64 gradients when
  given Float32 inputs. This prevents subsequent steps from taking advantage
  of OpenBLAS shortcuts at best, and causes type errors at worst.
  
  The following line converts all floats in the NN model to Float32. Dhairya
  claims this is a drastic but necessary stop gap measure. See more at
      https://github.com/FluxML/Flux.jl/issues/514
=#
Base.promote_rule(::Type{Float64}, ::Type{Float32}) = Float32

### Hyperparameters

In [ ]:
# How many .cifs should there be in the dataset?
#     _small: ~1000
#     _mid:   ~10000
#     _large: ~20000
#     _all:   ~400000
datasize = "_mid";

In [ ]:
# Hyperparameters
const train_frac = 0.8
const num_epochs = 8
const cutoff_radius = 8.0
const max_num_nbr = 12
decay_fcn=inverse_square

prop = "crystalsystem"
cif_dir = "../cif/"
graph_dir = "../newgraphs/$datasize/"

num_conv = 3
atom_fea_len = 32
pool_type = "mean"
crys_fea_len = 128
num_hidden_layers = 1
# lrs = exp10.(range(-3.2,stop=-2.8,length=5))
lr = 0.001

# features taken from ChemistryFeaturization.jl/data/pymatgen_atom_data.csv
features = ["Group", "Row", "X", "Atomic radius", "Block"]
num_bins = [18, 8, 10, 10, 4]
logspaced = [false, false, false, true, false]
num_features = sum(num_bins)

## Extracting and saving graphs from raw .cifs (Please skip)

The functions of this section is already recorded separately in graph_reader.jl

Optional if extracted, refined and saved graphs are available

In [8]:
labels = CSV.read("../labels/example.txt",
                  type="String", delim=", ")

# Discard cifs with missing labels for time being
labels = labels[labels[:crystalsystem] .!= "?", :]

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[8]:3
└ @ Core In[8]:3


,cif,crystalsystem
,String,String
1,1000000,monoclinic
2,1000001,orthorhombic
3,1000003,monoclinic
4,1000005,monoclinic
5,1000006,orthorhombic
6,1000017,trigonal
7,1000018,triclinic
8,1000019,monoclinic
9,1000020,monoclinic


In [ ]:
atom_feature_vecs = make_feature_vectors(features) # , num_bins, logspaced)
inputs = FeaturedGraph{SimpleWeightedGraph{Int32, Float32}, Array{Float32,2}}[]
cifs_without_graphs = Matrix(undef, 0, 2)
# rows_without_graphs = []

cif_root = labels[:, 1]

for (row, cif) in enumerate(cif_root)
    # path to the cif
    cif_path = string(cif_dir, cif[1], "/", cif[2:3], "/", 
                      cif[4:5], "/", cif, ".cif")
    try
        gr, atom_ids = @time build_graph(cif_path) # radius=radius, max_num_nbr=max_num_nbr, dist_decay_func=decay_func
        feature_mat = hcat([atom_feature_vecs[e] for e in atom_ids]...)
        input = FeaturedGraph(gr, feature_mat)
        push!(inputs, input)
    catch e
        println("unable to build graph at cif = ", cif)
        
        # Record cifs where graph building failed
        global cifs_without_graphs = [cifs_without_graphs; [row cif]]
        
        # remove this cif from list of cifs
        # labels = labels[labels[:cif] .!= cif, :]
    end
end


  2.778407 seconds (3.59 M allocations: 165.865 MiB, 2.30% gc time)
  0.240637 seconds (393.31 k allocations: 7.695 MiB)
  0.288019 seconds (444.02 k allocations: 8.546 MiB)
  0.664331 seconds (1.34 M allocations: 25.470 MiB)
  1.031110 seconds (1.34 M allocations: 25.462 MiB, 22.54% gc time)
  0.329687 seconds (744.75 k allocations: 14.037 MiB)
  0.715385 seconds (1.39 M allocations: 26.395 MiB)
  0.727235 seconds (1.37 M allocations: 25.990 MiB)
  0.427016 seconds (898.61 k allocations: 17.357 MiB)
  0.182913 seconds (355.04 k allocations: 6.770 MiB)
  0.091130 seconds (153.32 k allocations: 3.030 MiB)
  0.413393 seconds (789.00 k allocations: 15.425 MiB)
  1.387251 seconds (2.80 M allocations: 54.696 MiB)
  0.374230 seconds (631.21 k allocations: 12.261 MiB)
  0.326812 seconds (571.50 k allocations: 10.930 MiB)
  0.426926 seconds (698.83 k allocations: 13.361 MiB)
  0.279279 seconds (540.51 k allocations: 10.312 MiB)
  0.151795 seconds (339.63 k allocations: 6.524 MiB)
  0.486622 se

/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Structure has implicit hydrogens defined, parsed structure unlikely to be suitable for use in calculations unless hydrogens added.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1012: UserWarning: Some occupancies ([0.188, 0.312, 1.5, 1.0, 1.0]) sum to > 1! If they are within the tolerance, they will be rescaled.
  warnings.warn(msg)
/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/py

0.396361 seconds (821.00 k allocations: 15.737 MiB)
  0.707247 seconds (1.33 M allocations: 25.491 MiB)
  0.337874 seconds (676.67 k allocations: 12.975 MiB)
  0.204167 seconds (407.19 k allocations: 7.809 MiB)
  0.173767 seconds (319.31 k allocations: 6.082 MiB)
  0.550247 seconds (294.30 k allocations: 5.723 MiB, 32.81% gc time)
  0.719690 seconds (1.47 M allocations: 28.203 MiB)
  0.791705 seconds (1.56 M allocations: 29.746 MiB)
  0.913231 seconds (946.22 k allocations: 17.773 MiB, 21.96% gc time)
  0.329133 seconds (678.56 k allocations: 13.256 MiB)
unable to build graph at cif = 1000353
  0.638009 seconds (1.24 M allocations: 23.966 MiB)
  0.145677 seconds (254.37 k allocations: 5.036 MiB)
  0.603144 seconds (1.20 M allocations: 23.257 MiB)
  0.073978 seconds (132.82 k allocations: 2.530 MiB)
  0.930145 seconds (1.14 M allocations: 22.009 MiB, 17.60% gc time)
  0.831622 seconds (1.43 M allocations: 27.481 MiB)
  2.257750 seconds (3.99 M allocations: 79.177 MiB, 4.80% gc time)
  0

/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Structure has implicit hydrogens defined, parsed structure unlikely to be suitable for use in calculations unless hydrogens added.
Some occupancies ([1.0, 1.0, 1.5, 1.0, 1.0, 1.0]) sum to > 1! If they are within the tolerance, they will be rescaled.
Species occupancies sum to more than 1!
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1012: UserWarning: Some occupancies ([1.1667, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]) sum to > 1! If they are within the tolerance, they will be rescaled.
  warnings.warn(msg)
/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1117: UserWarning: data_1000353
loop_
 _publ_author_name
  'Crosnier-Lopez, M P'
  'Duroy, H'
  'Fourquet, J L'
_publ_secti

0.336286 seconds (586.84 k allocations: 11.369 MiB)
unable to build graph at cif = 1000365
  0.281309 seconds (584.62 k allocations: 11.329 MiB)
unable to build graph at cif = 1000366
  0.654182 seconds (582.17 k allocations: 11.282 MiB, 24.78% gc time)
unable to build graph at cif = 1000367
  0.273376 seconds (576.78 k allocations: 11.187 MiB)
unable to build graph at cif = 1000368
  0.726610 seconds (306.78 k allocations: 5.956 MiB)
  0.749438 seconds (606.22 k allocations: 11.951 MiB)
  1.112625 seconds (1.30 M allocations: 26.423 MiB)
  0.239936 seconds (153.92 k allocations: 2.925 MiB)
  0.091452 seconds (137.42 k allocations: 2.594 MiB)
unable to build graph at cif = 1000374
  0.828834 seconds (830.95 k allocations: 16.128 MiB)
  0.354303 seconds (655.79 k allocations: 12.647 MiB)
  1.109996 seconds (1.54 M allocations: 29.461 MiB, 14.53% gc time)
  0.205710 seconds (287.00 k allocations: 5.671 MiB)
  0.169452 seconds (303.36 k allocations: 5.755 MiB)
  0.192430 seconds (366.11 k

...

  0.191386 seconds (269.35 k allocations: 5.116 MiB)
  0.197008 seconds (257.71 k allocations: 5.030 MiB)
  0.166011 seconds (267.38 k allocations: 5.197 MiB)
  0.391035 seconds (637.29 k allocations: 12.404 MiB)
  0.339489 seconds (639.14 k allocations: 12.424 MiB)
  0.328311 seconds (697.51 k allocations: 13.471 MiB)
  0.092107 seconds (169.59 k allocations: 3.372 MiB)
  0.038516 seconds (54.44 k allocations: 1.038 MiB)
  0.105388 seconds (189.69 k allocations: 3.685 MiB)
  0.052206 seconds (98.34 k allocations: 1.927 MiB)
  0.127414 seconds (262.44 k allocations: 5.050 MiB)
unable to build graph at cif = 1008113
  0.052045 seconds (83.91 k allocations: 1.624 MiB)
  0.477946 seconds (962.71 k allocations: 18.189 MiB)
unable to build graph at cif = 1008115
  0.226345 seconds (260.45 k allocations: 5.062 MiB)
unable to build graph at cif = 1008116
  0.184334 seconds (313.64 k allocations: 6.113 MiB)
  0.049789 seconds (57.41 k allocations: 1.161 MiB)
  0.069068 seconds (43.57 k allocat

/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1121: UserWarning: Issues encountered while parsing CIF: Structure has implicit hydrogens defined, parsed structure unlikely to be suitable for use in calculations unless hydrogens added.
Some occupancies ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 9.0]) sum to > 1! If they are within the tolerance, they will be rescaled.
Species occupancies sum to more than 1!
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))
/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1012: UserWarning: Some occupancies ([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0]) sum to > 1! If they are within the tolerance, they will be rescaled.
  warnings.warn(msg)
/home/qingyanz/miniconda3/envs/pymatgen/lib/python3.8/site-packages/pymatgen/io/cif.py:1117: UserWarning: data_1008343
loop_
 _publ_author_name

0.213816 seconds (420.21 k allocations: 8.158 MiB)
unable to build graph at cif = 1008357
  2.187904 seconds (4.40 M allocations: 83.089 MiB)
unable to build graph at cif = 1008358
unable to build graph at cif = 1008359
  0.157344 seconds (325.42 k allocations: 6.210 MiB)
  0.347467 seconds (552.91 k allocations: 10.356 MiB)
unable to build graph at cif = 1008361
  0.242848 seconds (552.98 k allocations: 10.357 MiB)
unable to build graph at cif = 1008362
  1.832096 seconds (3.51 M allocations: 67.619 MiB)
unable to build graph at cif = 1008363
  0.427196 seconds (829.81 k allocations: 16.076 MiB)
  0.616729 seconds (1.14 M allocations: 24.032 MiB)
  1.068512 seconds (2.10 M allocations: 40.522 MiB)
  1.046510 seconds (2.09 M allocations: 40.286 MiB)
  0.595135 seconds (1.27 M allocations: 24.376 MiB)
  0.336883 seconds (613.51 k allocations: 11.728 MiB)
unable to build graph at cif = 1008370
  0.468046 seconds (797.88 k allocations: 15.262 MiB)
  0.091461 seconds (132.05 k allocations:

Excessive output truncated after 524297 bytes.

  0.108227 seconds (150.66 k

In [8]:
# remove cifs without feasible graphs
labels = labels[setdiff(1:end, rows_without_graphs), :]

UndefVarError: UndefVarError: rows_without_graphs not defined

## Reading saved graphs


In [ ]:
cifs_without_graphs = readdlm("../graphs/cifs_without_graphs$datasize.csv", ',', Int32);
labels = CSV.File("../labels/example$datasize.txt", delim=", ") |> DataFrame;

In [ ]:
# Remove cifs with corrupted or out-of-scope labels
label_names = ["triclinic",
               "monoclinic",
               "orthorhombic",
               "tetragonal",
               "trigonal",
               "hexagonal",
               "cubic"];

labels = labels[labels[:crystalsystem] .∈ Ref(label_names), :];

In [ ]:
# remove cifs without feasible graphs
labels = labels[setdiff(1:end, cifs_without_graphs[:, 1]), :];
cif_roots = string.(labels[:, 1]);

In [ ]:
inputs = AtomGraph[]
out_of_scope_cifs = []

for (row, cif) in enumerate(cif_roots)
    try
        input = deserialize(string(graph_dir, cif, ".jls"))
        push!(inputs, input)
    catch e
        push!(out_of_scope_cifs, [row cif])
        @info("graph building failed at cif = $cif")
    end
    
    if iszero(row % 100)
        @info("milestone: $row cifs processed ")
    end
end

In [ ]:
inputs[2]

In [29]:
#=
atom_feature_vecs = make_feature_vectors(Symbol.(features), nbins=num_bins, logspaced=logspaced)
element_lists = Array{String}[]
# inputs = FeaturedGraph{SimpleWeightedGraph{Int64, Float32}, Array{Float32,2}}[]
inputs = AtomGraph{SimpleWeightedGraph{Int32, Float32}}[]
cifs_beyond_scope = Matrix(undef, 0, 2)
=#

┌ Info: 16 elements were dropped so that all features are defined.
└ @ ChemistryFeaturization /home/qingyanz/.julia/packages/ChemistryFeaturization/5gGQK/src/atomfeat.jl:213


TypeError: TypeError: in Type{...} expression, expected UnionAll, got Type{AtomGraph}

### Old Graph Reader

In [12]:
function graph_reader!(inputs, element_lists, cifs_beyond_scope, atom_feature_vecs, datasize, cif_roots)
    prop = "crystalsystem"
    cif_dir = "../cif/"
    graph_dir = "../graphs/"
    
    for (row, cif) in enumerate(cif_roots)
        # path to the cif
        gr_path = string(graph_dir, "grwts", datasize, "/", cif, ".txt")
        el_path = string(graph_dir, "ellists", datasize, "/", cif, ".txt")

        try
            gr = SimpleWeightedGraph(Float32.(readdlm(gr_path)))
                    # restore to Float32
            els = readdlm(el_path)

            feature_mat = hcat([atom_feature_vecs[e] for e in els]...)
            input = FeaturedGraph(gr, feature_mat)
            push!(inputs, input)
            push!(element_lists, els)
        catch e
            println("unable to build graph at cif = ", cif)

            cifs_beyond_scope = vcat(cifs_beyond_scope, [row cif])
        end
    end
    
    println("Graph reading completed for ", length(cif_roots), " graphs")
    
    return inputs, element_lists, cifs_beyond_scope
end

inputs, element_lists, cifs_beyond_scope = graph_reader!(inputs, element_lists, 
                                                         cifs_beyond_scope, 
                                                         atom_feature_vecs,
                                                         datasize,
                                                         labels[:, 1])

labels = labels[setdiff(1:end, cifs_beyond_scope[:, 1]), :]
cif_roots = labels[:, 1];
y = labels[:, 2]

unable to build graph at cif = 1000365
unable to build graph at cif = 1000366
unable to build graph at cif = 1000367
unable to build graph at cif = 1000368
unable to build graph at cif = 1001038
unable to build graph at cif = 1001047
unable to build graph at cif = 1001094
unable to build graph at cif = 1001128
unable to build graph at cif = 1001132
unable to build graph at cif = 1001143
unable to build graph at cif = 1001150
unable to build graph at cif = 1001174
unable to build graph at cif = 1001183
unable to build graph at cif = 1001207
unable to build graph at cif = 1001214
unable to build graph at cif = 1001215
unable to build graph at cif = 1001217
unable to build graph at cif = 1001226
unable to build graph at cif = 1001329
unable to build graph at cif = 1001332
unable to build graph at cif = 1001337
unable to build graph at cif = 1001338
unable to build graph at cif = 1001339
unable to build graph at cif = 1001340
unable to build graph at cif = 1001341
unable to build graph at 

8411-element Array{String,1}:
 "triclinic"
 "triclinic"
 "orthorhombic"
 "tetragonal"
 "tetragonal"
 "tetragonal"
 "tetragonal"
 "tetragonal"
 "monoclinic"
 "cubic"
 "orthorhombic"
 "monoclinic"
 "orthorhombic"
 ⋮
 "monoclinic"
 "triclinic"
 "monoclinic"
 "monoclinic"
 "triclinic"
 "triclinic"
 "orthorhombic"
 "triclinic"
 "orthorhombic"
 "monoclinic"
 "monoclinic"
 "triclinic"

## Rearranging & Setting Up Inputs

In [13]:
num_pts = 500 # size(labels)[1] # for now
num_train = Int32(round(train_frac * num_pts))
num_test = num_pts - num_train

100

In [14]:
# Sample from y
indices = shuffle(1:size(labels,1))[1:num_pts]
ysample = y[indices]
input   = inputs[indices]
cif_roots_sample = cif_roots[indices];

In [15]:
ycat = collect(Set(ysample)) # Categorization of y
ycat_len = length(ycat)
output = Flux.onehotbatch(ysample, ycat)
# output = Flux.onehotbatch(ycat, ysample)

7×500 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  1  0  0  0  0  0  0  0  0  1  0  0  …  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0     0  0  0  0  1  0  1  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  1  1  1  1  1  0  1  0  0  0  …  0  1  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1     0  0  1  0  0  1  0  0  0  0  0  0

In [16]:
train_output = output[:, 1:num_train];
test_output  = output[:, num_train+1:end];
train_input  = input[1:num_train];
test_input   = input[num_train+1:end];
train_y      = ysample[1:num_train];
test_y       = ysample[num_train+1:end];
train_data   = zip(train_input, train_output);

In [17]:
typeof(train_data)

Base.Iterators.Zip{Tuple{Array{FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}},1},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}

In [18]:
# train_input = reshape(train_input, 1, length(train_input))
# test_input  = reshape(test_input, 1, length(test_input))

In [19]:
# train_data   = zip(train_input, train_output);

In [20]:
#=
train_output = output[1:num_train, :];
test_output = output[num_train+1:end, :];
train_input = input[1:num_train];
test_input = input[num_train+1:end];
train_data = zip(train_input, train_output);
=#

## Training

In [21]:
# Average pool
model = Chain(CGCNConv(num_features=>atom_fea_len), 
              [CGCNConv(atom_fea_len=>atom_fea_len) for i in 1:num_conv-1]..., 
              CGCNMeanPool(crys_fea_len, 0.1), 
              [Dense(crys_fea_len, crys_fea_len, softplus) for i in 1:num_hidden_layers-1]...,
              Dense(crys_fea_len, ycat_len), softmax);

#= 
# Max pool
model = Chain([CGCNConv(num_features=>num_features) for i in 1:num_conv]...,
              CGCNMaxPool(crys_fea_len, 0.1),
              [Dense(crys_fea_len, crys_fea_len, softplus) for i in 1:num_hidden_layers-1]..., 
              Dense(crys_fea_len, ycat_len), softmax);
=#

Chain(CGCNConv{Float32,typeof(softplus)}(Float32[0.03173602 -0.19352737 … 0.11125023 -0.00021721053; 0.24719378 0.13241613 … 0.26183918 0.18714796; … ; -0.14110063 -0.26994175 … 0.001628434 -0.14969423; 0.12076628 0.07879209 … 0.035923515 -0.18578647], Float32[0.05132405 0.25218472 … 0.20425555 -0.047553178; 0.02788866 0.0017774762 … -0.021772003 -0.2643269; … ; 0.0547991 0.09270562 … 0.10258458 0.014285849; 0.24016088 0.16484739 … 0.0026531545 0.08512125], Float32[0.37622014; 0.06400682; … ; 0.10393234; 0.010236247], NNlib.softplus), CGCNConv{Float32,typeof(softplus)}(Float32[-0.08574068 0.19484377 … 0.2523907 -0.05937312; -0.28339043 -0.1662954 … -0.12930532 -0.15053423; … ; 0.2869539 -0.17145953 … 0.14624618 -0.16313155; 0.04119067 -0.0703075 … -0.11050937 0.21210189], Float32[-0.2736572 -0.16474845 … 0.09944557 0.17449014; 0.040252835 -0.080837965 … 0.032895405 -0.04994985; … ; -0.14781116 0.19550866 … 0.1355976 -0.21073686; 0.17703588 -0.26603347 … -0.005452406 0.03458398], Float3

In [22]:
# Kullback-Leiber & Cross Entropy
loss_kl(x, y) = Flux.kldivergence(model(x), [y])
loss_ce(x, y) = Flux.logitcrossentropy(model(x), [y])
# loss_mse(x, y) = Flux.mse(model(x), y)
evalcb() = @show(mean(loss_kl.(test_input, test_output)))
start_err = evalcb()

println("Training started with $num_pts data points")
Dates.now()

DimensionMismatch: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 100 and 7")

In [23]:
test_input

100-element Array{FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}},1}:
 FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}}(Base.RefValue{SimpleWeightedGraph{Int64,Float32}}({176, 1295} undirected simple Int64 graph with Float32 weights), Base.RefValue{Array{Float32,2}}(Float32[1.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]))
 FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}}(Base.RefValue{SimpleWeightedGraph{Int64,Float32}}({152, 1091} undirected simple Int64 graph with Float32 weights), Base.RefValue{Array{Float32,2}}(Float32[1.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]))
 FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}}(Base.RefValue{SimpleWeightedGraph{Int64,Float32}}({124, 760} undirected simple Int64 graph with Float32 weights), Base.RefValue{Array{Float32,2}}(Float32[1.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [24]:
test_output

7×100 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  1  0  0     0  0  0  0  1  0  1  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0     1  0  0  0  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  1  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 1  0  1  1  0  0  1  0  0  0  0  0  0  …  0  1  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  1  1     0  0  1  0  0  1  0  0  0  0  0  0

In [25]:
loss_lce(x, y) = Flux.logitcrossentropy(model(x), y, dims=2)
loss_kl(x, y) = Flux.kldivergence(model(x), [y])
# loss_kl.(test_input, test_output)

loss_kl (generic function with 1 method)

In [26]:
train_data

Base.Iterators.Zip{Tuple{Array{FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}},1},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}((FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}}[FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}}(Base.RefValue{SimpleWeightedGraph{Int64,Float32}}({28, 195} undirected simple Int64 graph with Float32 weights), Base.RefValue{Array{Float32,2}}(Float32[1.0 1.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])), FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}}(Base.RefValue{SimpleWeightedGraph{Int64,Float32}}({3, 4} undirected simple Int64 graph with Float32 weights), Base.RefValue{Array{Float32,2}}(Float32[0.0 0.0 0.0; 0.0 0.0 0.0; … ; 1.0 0.0 0.0; 0.0 0.0 0.0])), FeaturedGraph{SimpleWeightedGraph{Int64,Float32},Array{Float32,2}}(Base.RefValue{SimpleWeightedGraph{Int64,Float32}}({40, 221} undirected simple Int64 graph with Float32 weights), Base.RefValue{Array{Floa

In [27]:
# train
opt = ADAM(lr)
_, train_time, mem, _, _ = @timed @epochs num_epochs Flux.train!(loss_lce, params(model), train_data, opt) #, cb=Flux.throttle(evalcb, 5))

┌ Info: Epoch 1
└ @ Main /home/qingyanz/.julia/packages/Flux/Fj3bt/src/optimise/train.jl:121


MethodError: MethodError: no method matching logitcrossentropy(::Array{Float32,2}, ::Bool; dims=2)
Closest candidates are:
  logitcrossentropy(::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T, !Matched::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T; weight) at /home/qingyanz/.julia/packages/Flux/Fj3bt/src/layers/stateless.jl:105 got unsupported keyword argument "dims"

In [28]:
X_test = model.(test_input)

100-element Array{Array{Float32,2},1}:
 [0.061649453; 0.13517928; … ; 0.25871778; 0.042253505]
 [0.061248265; 0.13500965; … ; 0.2618709; 0.0417964]
 [0.0611949; 0.13562983; … ; 0.26209798; 0.041898437]
 [0.06251065; 0.13747387; … ; 0.25826874; 0.043234907]
 [0.06278005; 0.13948047; … ; 0.25963482; 0.042082597]
 [0.061252486; 0.1351068; … ; 0.26177552; 0.041819703]
 [0.062101558; 0.13631447; … ; 0.2589227; 0.042825367]
 [0.061041076; 0.13496155; … ; 0.26163667; 0.04204142]
 [0.061833233; 0.13528429; … ; 0.25972578; 0.04220068]
 [0.061349228; 0.13496582; … ; 0.2615614; 0.041959923]
 [0.06290878; 0.13696401; … ; 0.25848335; 0.043112587]
 [0.061090335; 0.13466759; … ; 0.26139587; 0.04199078]
 [0.0610032; 0.13537961; … ; 0.2625462; 0.041778855]
 ⋮
 [0.061832573; 0.13690469; … ; 0.25998333; 0.042557083]
 [0.061324462; 0.13491307; … ; 0.26171568; 0.041940242]
 [0.061187252; 0.13539208; … ; 0.26196662; 0.041844018]
 [0.062189072; 0.1355038; … ; 0.2583612; 0.04296159]
 [0.061896976; 0.13477324;

In [29]:
hcat(X_test...)

7×100 Array{Float32,2}:
 0.0616495  0.0612483  0.0611949  …  0.0617031  0.0623036  0.0626984
 0.135179   0.13501    0.13563       0.134229   0.135299   0.136259
 0.194666   0.195656   0.195513      0.195606   0.194394   0.196701
 0.0536004  0.053138   0.0536052     0.0524686  0.0538898  0.0532667
 0.253934   0.251281   0.25006       0.254029   0.25368    0.250677
 0.258718   0.261871   0.262098   …  0.259971   0.257225   0.257163
 0.0422535  0.0417964  0.0418984     0.0419928  0.0432088  0.0432352

In [29]:
end_err = evalcb()
end_ce = @show(mean(loss_ce.(test_input, test_output)))

DimensionMismatch: DimensionMismatch("arrays could not be broadcast to a common size; got a dimension with lengths 100 and 8")

In [ ]:
# write results
try
    writedlm("../out/raw_predictions_datasize_$(num_pts)_train.csv", model.(train_input))
    writedlm("../out/raw_predictions_datasize_$(num_pts)_test.csv", model.(test_input))
catch e
    @info("Unable to write raw results")
end

ycat_dict = Dict(zip(1:ycat_len, ycat))
test_pred = model.(test_input)
test_pred_indices = mapslices(argmax, test_pred, dims=1)

correct_count = 0
prediction_labels = Matrix(undef, 0, 3)

function prediction_stats!(pred_indices, pred_dict, y, cif_roots,
                correct_count, prediction_labels)
    for (i, index) in enumerate(pred_indices)
        pred = pred_dict[index]
        label = y[i]
        cif_name = cif_roots[num_train+1:end][i]

        if pred == label
            correct_count += 1
        end

        prediction_labels = vcat(prediction_labels, [cif_name pred label])
    end
    
    return correct_count, prediction_labels
end

correct_count, prediction_labels = prediction_stats!(test_pred_indices, ycat_dict, 
                                                     test_y, cif_roots_sample,
                                                     correct_count, prediction_labels)

writedlm("../out/predictions_datasize_$(num_pts)_test.csv", prediction_labels)

accuracy = correct_count / num_test
println("The accuracy of our prediction is $accuracy")



## The above content condensed into a function

In [15]:
function cgcnn_train_long(args)
    num_pts, num_conv, atom_fea_len, pool_type, crys_fea_len, num_hidden_layers, lr, features, num_bins, logspaced = args
    cutoff_radius=8.0
    max_num_nbr=12
    decay_fcn=inverse_square    
    train_frac = 0.8
    num_epochs = 30
    num_epochs = 10

    prop = "band_gap"
    cif_dir = "../cif/"
    # id = 
    
    labels = CSV.read("../labels/example.txt",
                      type="String", delim=", ")

    # Discard cifs with missing labels for time being
    labels = labels[labels[:crystalsystem] .!= "?", :]
            
    atom_feature_vecs = make_feature_vectors(features, num_bins, logspaced)
    inputs = FeaturedGraph{SimpleWeightedGraph{Int32, Float32}, Array{Float32,2}}[]
    cifs_without_graphs = []
    rows_without_graphs = []
    
    cif_root = labels[:, 1]

    for row, cif in enumerate(cif_root)
        # path to the cif
        cif_path = string(cif_dir, cif[1], "/", cif[2:3], "/", 
                          cif[4:5], "/", cif, ".cif")
        try
            gr, atom_ids = @time build_graph(cif_path)
            feature_mat = hcat([atom_feature_vecs[e] for e in atom_ids]...)
            input = FeaturedGraph(gr, feature_mat)
            push!(inputs, input)
        catch e
            println("unable to build graph at cif = ", cif)

            # Record cifs where graph building failed
            push!(cifs_without_graphs, cif)
            push!(rows_without_graphs, row)
        end    
    end
            
    # remove cifs without feasible graphs
    labels = labels[setdiff(1:end, rows_without_graphs), :]
    
    num_pts = size(inputs)[1] # for now
    num_train = Int32(round(train_frac * num_pts))
    num_test = num_pts - num_train
            
    cif_root = labels[:, 1]
    y = labels[:, 2]

    # Sample from y
    indices = shuffle(1:size(labels,1))[1:num_pts]
    ysample = y[indices]
    ycat = collect(Set(ysample)) # Categorization of y
    ycat_len = length(ycat)
    output = Flux.onehotbatch(ysample, ycat)
            
    train_output = output[:, 1:num_train]
    test_output = output[:, num_train+1:end]
    train_input = inputs[1:num_train]
    test_input = inputs[num_train+1:end]
    train_data = zip(train_input, train_output)
    
    loss(x,y) = Flux.mse(model(x), y)
    evalcb() = @show(mean(loss.(test_input, test_output)))
    start_err = evalcb()
    
    opt = ADAM(lr)
    _, train_time, mem, _, _ = @timed @epochs num_epochs Flux.train!(loss, params(model), train_data, opt, cb=Flux.throttle(evalcb, 5))

    end_err = evalcb()

    loss_mae(x,y) = Flux.mae(model(x),y)
    end_mae = mean(loss_mae.(test_input, test_output))
    
    start_err, end_err, end_mae, train_time
end

LoadError: syntax: invalid iteration specification